In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../data'
group_dir_ = '../../data/groups/low'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [11]:
all_user = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
predictions = pd.read_pickle(os.path.join(dir_, 'post_filtering (count)', str(0.005) + '_prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

In [3]:
predictions[:5]

,uid,tid,rating
6,0,48,2.821581
11,0,70,3.864443
14,0,83,3.358146
32,0,176,2.833267
91,0,537,2.690027


In [4]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,4.75,15
37,0,15,2.50,3
45,0,22,3.50,6
64,0,28,2.75,4
75,0,33,0.25,1


In [5]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [6]:
num_user = len(predictions['uid'].unique())
num_user

953

In [7]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [12]:
top_n_size = 50
top_n_lists_g = []
top_n_items_g = []
for groups in groups_n:

    group_size = len(groups[0])
    top_n_lists = []
    top_n_items = pd.DataFrame()

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 
        
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)

        if len(top_n_items) == 0:
            top_n_items = rating_table
        else:
            top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()

    top_n_lists_g.append(top_n_lists)   
    top_n_items_g.append(top_n_items)

COV = []    
NOV = []
HitNOV = []
HitCOV = []
for i in range(len(groups_n)):
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    top_n_items = top_n_items_g[i]

    nov = 0
    hitNov = 0
    hits = np.zeros(len(df['tid'].unique()), dtype=bool)
    for j in (range(len(groups))):
        group = groups[j]
        top_n_list = top_n_lists[j]

        for k in range(len(group)):
            for l in range(top_n_size):
                uid = group[k]
                tid = top_n_list[l]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                nov += (1-pop_dict[tid])
                if len(t) > 0:
                    hits[int(tid)] = True
                    hitNov += (1-pop_dict[tid])
    NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
    COV.append(len(top_n_items)/len(df['tid'].unique()))
    HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
    HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
print(NOV)
print(HitNOV)
print(COV)
print(HitCOV)

[0.6749571116724143, 0.6867672428016194, 0.6988351489058017, 0.7000454704442363, 0.7038772298006545, 0.7075158070331519, 0.7098757742951202]
[0.07917173643226066, 0.057686880885914504, 0.0451915275689357, 0.03723253521574612, 0.0314606651509852, 0.026088273391526254, 0.02259494634938903]
[0.00316690677616506, 0.002589374678708105, 0.0022910888701314362, 0.0018785659433764683, 0.001662784720150793, 0.0015739336282343385, 0.0016183591741925657]
[0.0026718792640590986, 0.0022276238044768256, 0.001834140397418241, 0.0014533500034905787, 0.00121852926056852, 0.0010915991292592992, 0.0010852526226938382]
